In [1]:
from dsc80_utils import *

# Lecture 5 – Exploratory Data Analysis and Data Cleaning

## DSC 80, Winter 2025

### Announcements 📣

- Lab 2 due tomorrow, **Wed, Jan 22**.
- Project 1 is due this **Friday, Jan 24**.

### Agenda 📆

- Dataset overview.
- Introduction to `plotly`.
- Exploratory data analysis and feature types.
- Data cleaning.
    - Data quality checks.
    - Missing values.
    - Transformations and timestamps.
    - Modifying structure.
- Investigating student-submitted questions!

## Dataset overview

### San Diego food safety

From [this article](https://inewsource.org/2023/02/09/san-diego-restaurants-food-safety-violations/) ([archive link](https://archive.ph/gz8BL)):

> In the last three years, one third of San Diego County restaurants have had at least one major food safety violation.

### 99% Of San Diego Restaurants Earn ‘A' Grades, Bringing Usefulness of System Into Question

From [this article](https://www.nbcsandiego.com/news/local/99-of-san-diego-restaurants-earn-a-grades-bringing-usefulness-of-system-into-question/25381/) ([archive link](https://archive.ph/yB6RU)):

> Food held at unsafe temperatures. Employees not washing their hands. Dirty countertops. Vermin in the kitchen. An expired restaurant permit.
> 
> Restaurant inspectors for San Diego County found these violations during a routine health inspection of a diner in La Mesa in November 2016. Despite the violations, the restaurant was awarded a score of 90 out of 100, the lowest possible score to achieve an ‘A’ grade.

### The data

- We downloaded the data about the 1000 restaurants closest to UCSD from [here](https://www.sandiegocounty.gov/content/sdc/deh/fhd/ffis/intro.html.html).
- We had to download the data as JSON files, then process it into DataFrames. You'll learn how to do this soon!
    - Until now, you've (largely) been presented with CSV files that `pd.read_csv` could load without any issues.
    - But there are many different formats and possible issues when loading data in from files.
    - See [Chapter 8 of Learning DS](https://learningds.org/ch/08/files_intro.html) for more.

In [2]:
rest_path = Path('data') / 'restaurants.csv'
insp_path = Path('data') / 'inspections.csv'
viol_path = Path('data') / 'violations.csv'

In [3]:
rest = pd.read_csv(rest_path)
insp = pd.read_csv(insp_path)
viol = pd.read_csv(viol_path)

<div class="alert alert-warning">
    <h3>Question 🤔 (Answer at <a href="https://dsc80.com/q">dsc80.com/q</a>)</h3>

Code: `dfs`
</div>
    
The first article said that one third of restaurants had at least one major safety violation.<br>
Which DataFrames and columns seem most useful to verify this?

In [4]:
rest.head(2)

business_id                         name               business_type  \
0  211898487641  MOBIL MART LA JOLLA VILLAGE  Pre-Packaged Retail Market   
1  211930769329                     CAFE 477      Low Risk Food Facility   

                                             address  ...    lat    long  \
0       3233 LA JOLLA VILLAGE DR, LA JOLLA, CA 92037  ...  32.87 -117.23   
1  8950 VILLA LA JOLLA DR, SUITE# B123, LA JOLLA,...  ...  32.87 -117.24   

  opened_date distance  
0  2002-05-05     0.62  
1  2023-07-24     0.64  

[2 rows x 12 columns]

In [5]:
rest.columns

Index(['business_id', 'name', 'business_type', 'address', 'city', 'zip',
       'phone', 'status', 'lat', 'long', 'opened_date', 'distance'],
      dtype='object')

In [6]:
insp.head(2)

custom_id   business_id  inspection_id  description  ...  \
0  DEH2002-FFPN-310012  211898487641        6886133          NaN  ...   
1  DEH2002-FFPN-310012  211898487641        6631228          NaN  ...   

  completed_date    status                                               link  \
0     2023-02-16  Complete  http://www.sandiegocounty.gov/deh/fhd/ffis/ins...   
1     2022-01-03  Complete  http://www.sandiegocounty.gov/deh/fhd/ffis/ins...   

                                         status_link  
0  http://www.sandiegocounty.gov/deh/fhd/ffis/ins...  
1  http://www.sandiegocounty.gov/deh/fhd/ffis/ins...  

[2 rows x 11 columns]

In [7]:
insp.columns

Index(['custom_id', 'business_id', 'inspection_id', 'description', 'type',
       'score', 'grade', 'completed_date', 'status', 'link', 'status_link'],
      dtype='object')

In [8]:
viol.head(2)

inspection_id           violation major_violation  \
0        6886133  Hot and Cold Water               Y   
1        6631228  Hot and Cold Water               N   

                      status      violation_text  \
0  Out of Compliance - Major  Hot and Cold Water   
1  Out of Compliance - Minor  Hot and Cold Water   

                                correction_type_link  \
0  http://www.sandiegocounty.gov/deh/fhd/ffis/vio...   
1  http://www.sandiegocounty.gov/deh/fhd/ffis/vio...   

                 violation_accela  \
0  21. Hot & cold water available   
1  21. Hot & cold water available   

                                                link  
0  http://www.sandiegocounty.gov/deh/fhd/ffis/vio...  
1  http://www.sandiegocounty.gov/deh/fhd/ffis/vio...

In [9]:
viol.columns

Index(['inspection_id', 'violation', 'major_violation', 'status',
       'violation_text', 'correction_type_link', 'violation_accela', 'link'],
      dtype='object')

### Introduction to `plotly`

### `plotly`

- We've used `plotly` in lecture briefly, and you even have to use it in Project 1 Question 13, but we haven't yet discussed it formally.

- It's a visualization library that enables **interactive** visualizations.

<center><img src="imgs/plotly.png" width=40%></center>

### Using `plotly`

There are a few ways we can use `plotly`:

- Using the `plotly.express` syntax.
    - `plotly` is very flexible, but it can be verbose; `plotly.express` allows us to make plots quickly.
    - See the [**documentation here**](https://plotly.com/python/plotly-express) – it's very rich (there are good examples for almost everything).

- By setting `pandas` plotting backend to `'plotly'` (by default, it's `'matplotlib'`) and using the DataFrame `plot` method.
    - The DataFrame `plot` method is how you created plots in DSC 10!

For now, we'll use `plotly.express` syntax; we've imported it in the `dsc80_utils.py` file that we import at the top of each lecture notebook.

### Initial plots

First, let's look at the distribution of inspection `'score'`s:

In [10]:
fig = px.histogram(insp['score'])
fig

Error importing optional module nbformat
Traceback (most recent call last):
  File "c:\miniforge3\envs\dsc80\Lib\site-packages\_plotly_utils\optional_imports.py", line 28, in get_module
    return import_module(name)
           ^^^^^^^^^^^^^^^^^^^
  File "c:\miniforge3\envs\dsc80\Lib\importlib\__init__.py", line 90, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<frozen importlib._bootstrap>", line 1387, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1360, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1331, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 935, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 999, in exec_module
  File "<frozen importlib._bootstrap>", line 488, in _call_with_frames_removed
  File "c:\miniforge3\envs\dsc80\Lib\site-packages\nbformat\__init__.py", line 14, in <module>
    from

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'bingroup': 'x',
              'hovertemplate': 'variable=score<br>value=%{x}<br>count=%{y}<extra></extra>',
              'legendgroup': 'score',
              'marker': {'color': '#1F77B4', 'pattern': {'shape': ''}},
              'name': 'score',
              'offsetgroup': 'score',
              'orientation': 'v',
              'showlegend': True,
              'type': 'histogram',
              'x': array([96, 98, 98, ...,  0,  0, 90]),
              'xaxis': 'x',
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'legend': {'title': {'text': 'variable'}, 'tracegroupgap': 0},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'value'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'count'}}}
})

How about the distribution of average inspection `'score'` per `'grade'`?

In [11]:
scores = (
    insp[['grade', 'score']]
    .dropna()
    .groupby('grade')
    .mean()
    .reset_index()
)
# x= and y= are columns of scores. Convenient!
px.bar(scores, x='grade', y='score')

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': 'grade=%{x}<br>score=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#1F77B4', 'pattern': {'shape': ''}},
              'name': '',
              'offsetgroup': '',
              'orientation': 'v',
              'showlegend': False,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['A', 'B'], dtype=object),
              'xaxis': 'x',
              'y': array([97.27, 81.91]),
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'legend': {'tracegroupgap': 0},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'grade'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'score'}}}
})

In [12]:
# Same as the above!
scores.plot(kind='bar', x='grade', y='score')

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': 'grade=%{x}<br>score=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#1F77B4', 'pattern': {'shape': ''}},
              'name': '',
              'offsetgroup': '',
              'orientation': 'v',
              'showlegend': False,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['A', 'B'], dtype=object),
              'xaxis': 'x',
              'y': array([97.27, 81.91]),
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'legend': {'tracegroupgap': 0},
               'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'grade'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'score'}}}
})

## Exploratory data analysis and feature types

### The data science lifecycle, revisited

<center>
    <img src="imgs/ds-lifecycle.svg" width=50%>
</center>

We're at the stage of **understanding the data**.

### Exploratory data analysis (EDA)

- Historically, data analysis was dominated by formal statistics, including tools like confidence intervals, hypothesis tests, and statistical modeling.

- In 1977, John Tukey [defined](https://search.worldcat.org/title/3058187) the term **exploratory data analysis**, which described a philosophy for proceeding about data analysis:

> Exploratory data analysis is actively incisive, rather than passively descriptive, with real emphasis on the discovery of the unexpected.

- Practically, EDA involves, among other things, computing summary statistics and drawing plots to understand the nature of the data at hand.

> The greatest gains from data come from surprises… The unexpected is best brought to our attention by **pictures**.

### Different feature types

<center><img src='imgs/data-types.png' width=90%></center>

<div class="alert alert-warning">
    <h3>Question 🤔 (Answer at <a href="https://dsc80.com/q">dsc80.com/q</a>)</h3>

Code: `types`
</div>
    
Determine the **feature type** of each of the following variables.
    
- `insp['score']`
- `insp['grade']`
- `viol['violation_accela']`
- `viol['major_violation']`
- `rest['business_id']`
- `rest['opened_date']`

</div>



In [ ]:
# numerical discrete
# categorical ordinal
# categorical ordinal
# categorical nominal
# categorical ordinal
# categorical nominal

In [14]:
# Your code goes here.
insp['score']

0       96
1       98
2       98
        ..
5176     0
5177     0
5178    90
Name: score, Length: 5179, dtype: int64

In [15]:
# pandas stores these as ints, but they're actually nominal.
rest['business_id']

0      211898487641
1      211930769329
2      211909057778
           ...     
997    211899338714
998    211942150255
999    211925713322
Name: business_id, Length: 1000, dtype: int64

In [16]:
# pandas stores these as strings, but they're actually numeric.
rest['opened_date']

0      2002-05-05
1      2023-07-24
2      2019-01-22
          ...    
997    2002-05-05
998    2016-11-03
999    2022-11-03
Name: opened_date, Length: 1000, dtype: object

## Data cleaning

### Four pillars of data cleaning

When loading in a dataset, to clean the data – that is, to prepare it for further analysis – we will:

1. Perform **data quality checks**.

2. Identify and handle **missing values**.

3. Perform **transformations**, including converting time series data to **timestamps**.

4. Modify **structure** as necessary.

## Data cleaning: Data quality checks

### Data quality checks

We often start an analysis by checking the quality of the data.

- Scope: Do the data match your understanding of the population? 
- Measurements and values: Are the values reasonable?
- Relationships: Are related features in agreement?
- Analysis: Which features might be useful in a future analysis? 

### Scope

Do the data match your understanding of the population?

We were told that we're only looking at the 1000 restaurants closest to UCSD, so the restaurants in `rest` should agree with that.

In [17]:
rest.sample(5)

business_id                                               name  \
520  211945290016                       SPIROS MEDITERRANEAN CUISINE   
604  211899336336                              LA JOLLA COUNTRY CLUB   
307  211948689206                                  THAI TEST KITCHEN   
237  211898622703  THE LODGE AT TORREY PINES - MAIN KITCHEN-AR VA...   
917  211924685833                                       THE BUTCHERY   

                business_type  \
520  Restaurant Food Facility   
604  Restaurant Food Facility   
307  Restaurant Food Facility   
237  Restaurant Food Facility   
917  Restaurant Food Facility   

                                              address  ...    lat    long  \
520  909 PROSPECT ST, SUITE# 100B, LA JOLLA, CA 92037  ...  32.85 -117.28   
604                  7301 HIGH ST, LA JOLLA, CA 92037  ...  32.84 -117.27   
307             10184 TELESIS CT, SAN DIEGO, CA 92121  ...  32.90 -117.21   
237       11480 N TORREY PINES RD, LA JOLLA, CA 92037  ...  32.90 -117.24   
917           3745 PASEO PL, SAN DIEGO, CA 92130-2569  ...  32.95 -117.23   

    opened_date distance  
520  2020-03-11     3.35  
604  2002-05-05     3.59  
307  2023-05-09     2.09  
237  2002-05-05     1.60  
917  2018-08-31     4.73  

[5 rows x 12 columns]

### Measurements and values

Are the values reasonable?

Do the values in the `'grade'` column match what we'd expect grades to look like?

In [18]:
insp['grade'].value_counts()

grade
A    2978
B      11
Name: count, dtype: int64

What kinds of information does the `insp` DataFrame hold?

In [19]:
insp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5179 entries, 0 to 5178
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   custom_id       5179 non-null   object 
 1   business_id     5179 non-null   int64  
 2   inspection_id   5179 non-null   int64  
 3   description     0 non-null      float64
 4   type            5179 non-null   object 
 5   score           5179 non-null   int64  
 6   grade           2989 non-null   object 
 7   completed_date  5179 non-null   object 
 8   status          5179 non-null   object 
 9   link            5179 non-null   object 
 10  status_link     5179 non-null   object 
dtypes: float64(1), int64(3), object(7)
memory usage: 445.2+ KB


What's going on in the `'address'` column of `rest`?

In [20]:
# Are there multiple restaurants with the same address?
rest['address'].value_counts()

address
5300 GRAND DEL MAR CT, SAN DIEGO, CA 92130         9
8657 VILLA LA JOLLA DR, LA JOLLA, CA 92037         8
4545 LA JOLLA VILLAGE DR, SAN DIEGO, CA 92122      8
                                                  ..
8825 VILLA LA JOLLA DR, LA JOLLA, CA 92037-1949    1
4360 GENESEE AVE, SAN DIEGO, CA 92117-4901         1
9420 ATHENA CIR, LA JOLLA, CA 92037-1387           1
Name: count, Length: 863, dtype: int64

In [21]:
# Keeps all rows with duplicate addresses.
(
    rest
    .groupby('address')
    .filter(lambda df: df.shape[0] >= 2)
    .sort_values('address')
)

business_id                               name  \
406  211899308875             NASEEMS BAKERY & KABOB   
402  211898699154                  HANAYA SUSHI CAFE   
401  211899558107              ARMANDOS MEXICAN FOOD   
..            ...                                ...   
575  211972411855           TARA HEATHER CAKE DESIGN   
344  211990537315  COMPASS GROUP FEDEX EXPRESS OLSON   
343  211976587262             CANTEEN - FED EX OLSON   

                  business_type  \
406    Restaurant Food Facility   
402    Restaurant Food Facility   
401    Restaurant Food Facility   
..                          ...   
575                     Caterer   
344  Pre-Packaged Retail Market   
343  Pre-Packaged Retail Market   

                                               address  ...    lat    long  \
406    10066 PACIFIC HEIGHTS BLVD, SAN DIEGO, CA 92121  ...  32.90 -117.19   
402    10066 PACIFIC HEIGHTS BLVD, SAN DIEGO, CA 92121  ...  32.90 -117.19   
401    10066 PACIFIC HEIGHTS BLVD, SAN DIEGO, CA 92121  ...  32.90 -117.19   
..                                                 ...  ...    ...     ...   
575  9932 MESA RIM RD, SUITE# A, SAN DIEGO, CA 9212...  ...  32.90 -117.18   
344            9999 OLSON DR, SAN DIEGO, CA 92121-2837  ...  32.89 -117.20   
343            9999 OLSON DR, SAN DIEGO, CA 92121-2837  ...  32.89 -117.20   

    opened_date distance  
406  2012-04-17     2.77  
402  2011-03-22     2.77  
401  2005-06-28     2.77  
..          ...      ...  
575  2014-04-24     3.51  
344  2022-10-19     2.27  
343  2020-07-31     2.27  

[213 rows x 12 columns]

In [22]:
# Does the same thing as above!
(
    rest[rest.duplicated(subset=['address'], keep=False)]
    .sort_values('address')
)

business_id                               name  \
406  211899308875             NASEEMS BAKERY & KABOB   
402  211898699154                  HANAYA SUSHI CAFE   
401  211899558107              ARMANDOS MEXICAN FOOD   
..            ...                                ...   
575  211972411855           TARA HEATHER CAKE DESIGN   
344  211990537315  COMPASS GROUP FEDEX EXPRESS OLSON   
343  211976587262             CANTEEN - FED EX OLSON   

                  business_type  \
406    Restaurant Food Facility   
402    Restaurant Food Facility   
401    Restaurant Food Facility   
..                          ...   
575                     Caterer   
344  Pre-Packaged Retail Market   
343  Pre-Packaged Retail Market   

                                               address  ...    lat    long  \
406    10066 PACIFIC HEIGHTS BLVD, SAN DIEGO, CA 92121  ...  32.90 -117.19   
402    10066 PACIFIC HEIGHTS BLVD, SAN DIEGO, CA 92121  ...  32.90 -117.19   
401    10066 PACIFIC HEIGHTS BLVD, SAN DIEGO, CA 92121  ...  32.90 -117.19   
..                                                 ...  ...    ...     ...   
575  9932 MESA RIM RD, SUITE# A, SAN DIEGO, CA 9212...  ...  32.90 -117.18   
344            9999 OLSON DR, SAN DIEGO, CA 92121-2837  ...  32.89 -117.20   
343            9999 OLSON DR, SAN DIEGO, CA 92121-2837  ...  32.89 -117.20   

    opened_date distance  
406  2012-04-17     2.77  
402  2011-03-22     2.77  
401  2005-06-28     2.77  
..          ...      ...  
575  2014-04-24     3.51  
344  2022-10-19     2.27  
343  2020-07-31     2.27  

[213 rows x 12 columns]

### Relationships

Are related features in agreement?

Do the `'address'`es and `'zip'` codes in `rest` match?

In [23]:
rest[['address', 'zip']]

address         zip
0         3233 LA JOLLA VILLAGE DR, LA JOLLA, CA 92037       92037
1    8950 VILLA LA JOLLA DR, SUITE# B123, LA JOLLA,...  92037-1704
2               6902 LA JOLLA BLVD, LA JOLLA, CA 92037       92037
..                                                 ...         ...
997       1234 TOURMALINE ST, SAN DIEGO, CA 92109-1856  92109-1856
998  12925 EL CAMINO REAL, SUITE# AA4, SAN DIEGO, C...       92130
999         2672 DEL MAR HEIGHTS RD, DEL MAR, CA 92014       92014

[1000 rows x 2 columns]

What about the `'score'`s and `'grade'`s in `insp`?

In [24]:
insp[['score', 'grade']]

score grade
0        96   NaN
1        98   NaN
2        98   NaN
...     ...   ...
5176      0   NaN
5177      0   NaN
5178     90     A

[5179 rows x 2 columns]

### Analysis

Which features might be useful in a future analysis?

- We're most interested in:
    - These columns in the `rest` DataFrame: `'business_id'`, `'name'`, `'address'`, `'zip'`, and `'opened_date'`.
    - These columns in the `insp` DataFrame: `'business_id'`, `'inspection_id'`, `'score'`, `'grade'`, `'completed_date'`, and `'status'`.
    - These columns in the `viol` DataFrame: `'inspection_id'`, `'violation'`, `'major_violation'`, `'violation_text'`, and `'violation_accela'`.

- Also, let's rename a few columns to make them easier to work with.

### 💡 Pro-Tip: Using `pipe`

When we manipulate DataFrames, it's best to define individual functions for each step, then use the `pipe` **method** to chain them all together.

The `pipe` DataFrame method takes in a function, which itself takes in a DataFrame and returns a DataFrame.

- In practice, we would add functions one by one to the top of a notebook, then `pipe` them all.
- For today, will keep re-running `pipe` to show data cleaning process.

In [25]:
def subset_rest(rest):
    return rest[['business_id', 'name', 'address', 'zip', 'opened_date']]

rest = (
    pd.read_csv(rest_path)
    .pipe(subset_rest)
)
rest

business_id                         name  \
0    211898487641  MOBIL MART LA JOLLA VILLAGE   
1    211930769329                     CAFE 477   
2    211909057778           VALLEY FARM MARKET   
..            ...                          ...   
997  211899338714     PACIFIC BEACH ELEMENTARY   
998  211942150255              POKEWAN DEL MAR   
999  211925713322   SAFFRONO LOUNGE RESTAURANT   

                                               address         zip opened_date  
0         3233 LA JOLLA VILLAGE DR, LA JOLLA, CA 92037       92037  2002-05-05  
1    8950 VILLA LA JOLLA DR, SUITE# B123, LA JOLLA,...  92037-1704  2023-07-24  
2               6902 LA JOLLA BLVD, LA JOLLA, CA 92037       92037  2019-01-22  
..                                                 ...         ...         ...  
997       1234 TOURMALINE ST, SAN DIEGO, CA 92109-1856  92109-1856  2002-05-05  
998  12925 EL CAMINO REAL, SUITE# AA4, SAN DIEGO, C...       92130  2016-11-03  
999         2672 DEL MAR HEIGHTS RD, DEL MAR, CA 92014       92014  2022-11-03  

[1000 rows x 5 columns]

In [26]:
# Same as the above – but the above makes it easier to chain more .pipe calls afterwards.
subset_rest(pd.read_csv(rest_path))

business_id                         name  \
0    211898487641  MOBIL MART LA JOLLA VILLAGE   
1    211930769329                     CAFE 477   
2    211909057778           VALLEY FARM MARKET   
..            ...                          ...   
997  211899338714     PACIFIC BEACH ELEMENTARY   
998  211942150255              POKEWAN DEL MAR   
999  211925713322   SAFFRONO LOUNGE RESTAURANT   

                                               address         zip opened_date  
0         3233 LA JOLLA VILLAGE DR, LA JOLLA, CA 92037       92037  2002-05-05  
1    8950 VILLA LA JOLLA DR, SUITE# B123, LA JOLLA,...  92037-1704  2023-07-24  
2               6902 LA JOLLA BLVD, LA JOLLA, CA 92037       92037  2019-01-22  
..                                                 ...         ...         ...  
997       1234 TOURMALINE ST, SAN DIEGO, CA 92109-1856  92109-1856  2002-05-05  
998  12925 EL CAMINO REAL, SUITE# AA4, SAN DIEGO, C...       92130  2016-11-03  
999         2672 DEL MAR HEIGHTS RD, DEL MAR, CA 92014       92014  2022-11-03  

[1000 rows x 5 columns]

Let's use `pipe` to keep (and rename) the subset of the columns we care about in the other two DataFrames as well.

In [27]:
def subset_insp(insp):
    return (
        insp[['business_id', 'inspection_id', 'score', 'grade', 'completed_date', 'status']]
        .rename(columns={'completed_date': 'date'})
    )

insp = (
    pd.read_csv(insp_path)
    .pipe(subset_insp)
)

In [28]:
def subset_viol(viol):
    return (
        viol[['inspection_id', 'violation', 'major_violation', 'violation_accela']]
        .rename(columns={'violation': 'kind',
                         'major_violation': 'is_major',
                         'violation_accela': 'violation'})
    )

viol = (
    pd.read_csv(viol_path)
    .pipe(subset_viol)
)

### Combining the restaurant data

Let's join all three DataFrames together so that we have all the data in a single DataFrame.

In [ ]:
def merge_all_restaurant_data():
    return (
        rest
        .merge(insp, on='business_id', how='left')
        .merge(viol, on='inspection_id', how='left')
    )

df = merge_all_restaurant_data()
df

<div class="alert alert-warning">
    <h3>Question 🤔 (Answer at <a href="https://dsc80.com/q">dsc80.com/q</a>)</h3>

Code: `lefts`
</div>
    
Why should the function above use two left joins? What would go wrong if we used other kinds of joins?



## Data cleaning: Missing values

### Missing values

Next, it's important to check for and handle missing values, as they can have a big effect on your analysis.

In [ ]:
insp[['score', 'grade']]

In [ ]:
# The proportion of values in each column that are missing.
insp.isna().mean()

In [ ]:
# Why are there null values here?
# insp['inspection_id'] and viol['inspection_id'] don't have any null values...
df[df['inspection_id'].isna()]

There are many ways of handling missing values, which we'll cover in an entire lecture next week. But a good first step is to check how many there are!

## Data cleaning: Transformations and timestamps

### Transformations and timestamps

From last class:

> A transformation results from performing some operation on every element in a sequence, e.g. a Series.

It's often useful to look at ways of transforming your data to make it easier to work with.

- Type conversions (e.g. changing the string `"$2.99"` to the number `2.99`).

- Unit conversion (e.g. feet to meters).

- Extraction (Getting `'vermin'` out of `'Vermin Violation Recorded on 10/10/2023'`).

### Creating timestamps

Most commonly, we'll parse dates into `pd.Timestamp` objects.

In [ ]:
# Look at the dtype!
insp['date']

In [ ]:
# This magical string tells Python what format the date is in.
# For more info: https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior
date_format = '%Y-%m-%d'
pd.to_datetime(insp['date'], format=date_format)

In [ ]:
# Another advantage of defining functions is that we can reuse this function
# for the 'opened_date' column in `rest` if we wanted to.
def parse_dates(insp, col):
    date_format = '%Y-%m-%d'
    dates = pd.to_datetime(insp[col], format=date_format)
    return insp.assign(**{col: dates})

insp = (
    pd.read_csv(insp_path)
    .pipe(subset_insp)
    .pipe(parse_dates, 'date')
)

# We should also remake df, since it depends on insp.
# Note that the new insp is used to create df!
df = merge_all_restaurant_data()

In [ ]:
# Look at the dtype now!
df['date']

### Working with timestamps

- We often want to adjust granularity of timestamps to see overall trends, or seasonality.
- Use the `resample` method in `pandas` ([documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects)).
    - Think of it like a version of `groupby`, but for timestamps.
    - For instance, `insp.resample('2W', on='date')` separates every two weeks of data into a different group.

In [ ]:
insp.resample('2W', on='date')['score'].mean()

In [ ]:
# Where are those numbers coming from?
insp[
    (insp['date'] >= pd.Timestamp('2020-01-05')) &
    (insp['date'] < pd.Timestamp('2020-01-19'))
]['score']

In [ ]:
(insp.resample('2W', on='date')
 .size()
 .plot(title='Number of Inspections Over Time')
)

### The `.dt` accessor

Like with Series of strings, `pandas` has a `.dt` accessor for properties of timestamps ([documentation](https://pandas.pydata.org/docs/user_guide/basics.html#basics-dt-accessors)).

In [ ]:
insp['date']

In [ ]:
insp['date'].dt.day

In [ ]:
insp['date'].dt.dayofweek

In [ ]:
dow_counts = insp['date'].dt.dayofweek.value_counts()
fig = px.bar(dow_counts)
fig.update_xaxes(tickvals=np.arange(7), ticktext=['Mon', 'Tues', 'Wed', 'Thurs', 'Fri', 'Sat', 'Sun'])

## Data cleaning: Modifying structure

### Reshaping DataFrames

We often **reshape** the DataFrame's structure to make it more convenient for analysis. For example, we can:

- Simplify structure by removing columns or taking a set of rows for a particular period of time or geographic area.
    - We already did this!

- Adjust granularity by aggregating rows together.
    - To do this, use `groupby` (or `resample`, if working with timestamps).

- Reshape structure, most commonly by using the DataFrame `melt` method to un-pivot a dataframe.

### Using `melt`

- The `melt` method is common enough that we'll give it a special mention.
- We'll often encounter pivot tables (esp. from government data), which we call *wide* data.
- The methods we've introduced work better with *long-form* data, or *tidy* data.
- To go from wide to long, `melt`.

<center><img src='imgs/wide-vs-long.svg' width=40%></center>

### Example usage of `melt`

In [ ]:
wide_example = pd.DataFrame({
    'Year': [2001, 2002],
    'Jan': [10, 130],
    'Feb': [20, 200],
    'Mar': [30, 340]
}).set_index('Year')
wide_example

In [ ]:
wide_example.melt(ignore_index=False)

## Exploration

<div class="alert alert-warning">
    <h3>Question 🤔 (Answer at <a href="https://dsc80.com/q">dsc80.com/q</a>)</h3>

Code: `qs`
</div>
    
What questions do you want me to try and answer with the data? I'll start with a single pre-prepared question, and then answer student questions until we run out of time.

### Example question: Can we rank restaurants by their number of violations? How about separately for each zip code?

And why would we want to do that? 🤔

## Summary, next time

### Summary

- Data cleaning is a necessary starting step in data analysis. There are four pillars of data cleaning:
    - Quality checks.
    - Missing values.
    - Transformations and timestamps.
    - Modifying structure.
- Approach EDA with an open mind, and draw lots of visualizations.

### Next time

Hypothesis and permutation testing. Some of this will be DSC 10 review, but we'll also push further! 